In [2]:
!git clone https://github.com/moaaztaha/Arabic-English-Translation-Transformers

Cloning into 'Arabic-English-Translation-Transformers'...
remote: Enumerating objects: 24, done.
remote: Counting objects: 100% (24/24), done.
remote: Compressing objects: 100% (22/22), done.
remote: Total 24 (delta 0), reused 24 (delta 0), pack-reused 0
Unpacking objects: 100% (24/24), done.


In [21]:
# modules
import random
import string
import re
from pathlib import Path
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

### Data Preprocessing 

In [24]:
en = pd.read_table('/content/Arabic-English-Translation-Transformers/data/eng/ac-test.en', delimiter='\\n', names=['en'])
ar = pd.read_table('/content/Arabic-English-Translation-Transformers/data/ara/test.en_ref.ar', delimiter='\\n', names=['ar'])
en['ar'] = ar['ar']
df = en.copy()
df.head()

/usr/local/lib/python3.7/dist-packages/pandas/io/parsers.py:767: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  return read_csv(**locals())


,en,ar
0,"THE COUNCIL OF THE EUROPEAN ECONOMIC COMMUNITY,",مجلس الجماعة الاقتصادية الأوروبية
1,Whereas the adoption of a common transport pol...,حيث أن اعتماد سياسة نقل مشتركة تنطوي من بين أم...
2,Article 1,المادة 1
3,3. The types of carriage listed in Annex II sh...,3. لا تخضع أنواع النقل المدرجة في الملحق الثان...
4,Member States shall inform the Commission of t...,تبلغ الدول الأعضاء المفوضية الأوروبية بالتدابي...


In [25]:
text_pairs = []
for idx, row in df.iterrows():
    # split sentences
    if '.' in row['en'] and '.' in row['ar'] and len(row['en'].split()) > 100:
        en_sents = row['en'].split('.')
        ar_sents = row['ar'].split('.')
    
        for en_sent, ar_sent in zip(en_sents, ar_sents):
            ar_sent = "[start] " + ar_sent + " [end]"
            text_pairs.append((en_sent, ar))
    else:
        en, ar = row['en'], row['ar']
        ar = "[start] " + ar + " [end]"
        text_pairs.append((en, ar))

In [26]:
for _ in range(2):
    print(random.choice(text_pairs))

('(8) In the light of new scientific knowledge, testing methods should be reviewed, including testing methods for analysing 4-amino azobenzene.', '[start] (8) في ضوء المعرفة العلمية الجديدة، ينبغي إعادة النظر في أساليب الاختبار، بما في ذلك أساليب الاختبار المخصصة لتحليل الأزوبنزين الأميني 4. [end]')
('Member States shall communicate to the Commission provisions laid down by law, regulation or administrative action which they have adopted for the application of legal acts of the Community relating to the common agricultural policy in so far as those acts have financial consequences for the Fund.', '[start] تقوم الدول الأعضاء بالإرسال إلى المفوضية جميع الأحكام التي نص عليها القانون واللوائح أو الإجراء الإداري الذي قاموا بالتصديق عليه لتطبيق الأعمال القانونية الخاصة بالمجموعة فيما يتعلق بالسياسة الزراعية المشتركة طالما أن هذه الأعمال لديها نتائج مالية خاصة بالصندوق. [end]')


In [27]:
random.shuffle(text_pairs)
num_val_samples = int(0.15 * len(text_pairs))
num_train_samples = len(text_pairs) -  num_val_samples
train_pairs = text_pairs[: num_train_samples]
val_pairs = text_pairs[num_train_samples: num_train_samples + num_val_samples]

In [28]:
print(f"{len(text_pairs)} total pairs")
print(f"{len(train_pairs)} training pairs")
print(f"{len(val_pairs)} validation pairs")

4262 total pairs
3623 training pairs
639 validation pairs


#### Vectorizing the text data 

In [34]:
strip_chars = string.punctuation
strip_chars = strip_chars.replace("[", "")
strip_chars = strip_chars.replace("]", "")


vocab_size = 25000
sequence_length = 100
batch_size = 128

def custom_standardization(input_string):
    return tf.strings.regex_replace(input_string, "[%s]" % re.escape(strip_chars), "")

eng_vectorization = TextVectorization(
            max_tokens=vocab_size, output_mode='int', output_sequence_length=sequence_length)

ar_vectorization = TextVectorization(
    max_tokens=vocab_size,
    output_mode="int",
    output_sequence_length=sequence_length + 1,
    standardize=custom_standardization)

eng_texts = [pair[0] for pair in text_pairs]
ar_texts = [pair[1] for pair in text_pairs]
eng_vectorization.adapt(eng_texts)
ar_vectorization.adapt(ar_texts)

In [35]:
def format_dataset(eng, ar):
    eng = eng_vectorization(eng)
    ar = ar_vectorization(ar)
    return ({"encoder_inputs": eng, "decoder_inputs": ar[:, :-1],}, ar[:, 1:])


def make_dataset(pairs):
    eng_texts, ar_texts = zip(*pairs)
    eng_texts = list(eng_texts)
    ar_texts = list(ar_texts)
    dataset = tf.data.Dataset.from_tensor_slices((eng_texts, ar_texts))
    dataset = dataset.batch(batch_size)
    dataset = dataset.map(format_dataset)
    return dataset.shuffle(2048).prefetch(16).cache()

In [36]:
train_ds = make_dataset(train_pairs)
val_ds = make_dataset(val_pairs)

In [37]:
for inputs, targets in train_ds.take(1):
    print(f'inputs["encoder_inputs"].shape: {inputs["encoder_inputs"].shape}')
    print(f'inputs["decoder_inputs"].shape: {inputs["decoder_inputs"].shape}')
    print(f"targets.shape: {targets.shape}")

inputs["encoder_inputs"].shape: (128, 100)
inputs["decoder_inputs"].shape: (128, 100)
targets.shape: (128, 100)


### Building the Model 

In [38]:
class TransformerEncoder(layers.Layer):
    def __init__(self, embed_dim, dense_dim, num_heads, **kwargs):
        super(TransformerEncoder, self).__init__(**kwargs)
        self.embed_dim = embed_dim
        self.dense_dim = dense_dim
        self.num_heads = num_heads
        self.attention = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim
        )
        self.dense_proj = keras.Sequential(
            [layers.Dense(dense_dim, activation="relu"), layers.Dense(embed_dim),]
        )
        self.layernorm_1 = layers.LayerNormalization()
        self.layernorm_2 = layers.LayerNormalization()
        self.supports_masking = True

    def call(self, inputs, mask=None):
        if mask is not None:
            padding_mask = tf.cast(mask[:, tf.newaxis, tf.newaxis, :], dtype="int32")
        attention_output = self.attention(
            query=inputs, value=inputs, key=inputs, attention_mask=padding_mask
        )
        proj_input = self.layernorm_1(inputs + attention_output)
        proj_output = self.dense_proj(proj_input)
        return self.layernorm_2(proj_input + proj_output)

    def get_config(self):

      config = super().get_config().copy()
      config.update({
          'embed_dim': self.embed_dim,
          'dense_dim': self.dense_dim,
          'num_heads': self.num_heads,
      })
      return config

class PositionalEmbedding(layers.Layer):
    def __init__(self, sequence_length, vocab_size, embed_dim, **kwargs):
        super(PositionalEmbedding, self).__init__(**kwargs)
        self.token_embeddings = layers.Embedding(
            input_dim=vocab_size, output_dim=embed_dim
        )
        self.position_embeddings = layers.Embedding(
            input_dim=sequence_length, output_dim=embed_dim
        )
        self.sequence_length = sequence_length
        self.vocab_size = vocab_size
        self.embed_dim = embed_dim
      
    def get_config(self):

      config = super().get_config().copy()
      config.update({
      'sequence_length': self.sequence_length,
      'vocab_size': self.vocab_size,
      'embed_dim': self.embed_dim,
      })
      return config

    def call(self, inputs):
        length = tf.shape(inputs)[-1]
        positions = tf.range(start=0, limit=length, delta=1)
        embedded_tokens = self.token_embeddings(inputs)
        embedded_positions = self.position_embeddings(positions)
        return embedded_tokens + embedded_positions

    def compute_mask(self, inputs, mask=None):
        return tf.math.not_equal(inputs, 0)


class TransformerDecoder(layers.Layer):
    def __init__(self, embed_dim, latent_dim, num_heads, **kwargs):
        super(TransformerDecoder, self).__init__(**kwargs)
        self.embed_dim = embed_dim
        self.latent_dim = latent_dim
        self.num_heads = num_heads
        self.attention_1 = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim
        )
        self.attention_2 = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim
        )
        self.dense_proj = keras.Sequential(
            [layers.Dense(latent_dim, activation="relu"), layers.Dense(embed_dim),]
        )
        self.layernorm_1 = layers.LayerNormalization()
        self.layernorm_2 = layers.LayerNormalization()
        self.layernorm_3 = layers.LayerNormalization()
        self.supports_masking = True


    def get_config(self):

      config = super().get_config().copy()
      config.update({
      'embed_dim': self.embed_dim,
      'latent_dim': self.latent_dim,
      'num_heads': self.num_heads,
      })
      return config

    def call(self, inputs, encoder_outputs, mask=None):
        causal_mask = self.get_causal_attention_mask(inputs)
        if mask is not None:
            padding_mask = tf.cast(mask[:, tf.newaxis, :], dtype="int32")
            padding_mask = tf.minimum(padding_mask, causal_mask)

        attention_output_1 = self.attention_1(
            query=inputs, value=inputs, key=inputs, attention_mask=causal_mask
        )
        out_1 = self.layernorm_1(inputs + attention_output_1)

        attention_output_2 = self.attention_2(
            query=out_1,
            value=encoder_outputs,
            key=encoder_outputs,
            attention_mask=padding_mask,
        )
        out_2 = self.layernorm_2(out_1 + attention_output_2)

        proj_output = self.dense_proj(out_2)
        return self.layernorm_3(out_2 + proj_output)

    def get_causal_attention_mask(self, inputs):
        input_shape = tf.shape(inputs)
        batch_size, sequence_length = input_shape[0], input_shape[1]
        i = tf.range(sequence_length)[:, tf.newaxis]
        j = tf.range(sequence_length)
        mask = tf.cast(i >= j, dtype="int32")
        mask = tf.reshape(mask, (1, input_shape[1], input_shape[1]))
        mult = tf.concat(
            [tf.expand_dims(batch_size, -1), tf.constant([1, 1], dtype=tf.int32)],
            axis=0,
        )
        return tf.tile(mask, mult)

In [39]:
embed_dim = 256
latent_dim = 2048
num_heads = 8

encoder_inputs = keras.Input(shape=(None,), dtype="int64", name="encoder_inputs")
x = PositionalEmbedding(sequence_length, vocab_size, embed_dim)(encoder_inputs)
encoder_outputs = TransformerEncoder(embed_dim, latent_dim, num_heads)(x)
encoder = keras.Model(encoder_inputs, encoder_outputs)

decoder_inputs = keras.Input(shape=(None,), dtype="int64", name="decoder_inputs")
encoded_seq_inputs = keras.Input(shape=(None, embed_dim), name="decoder_state_inputs")
x = PositionalEmbedding(sequence_length, vocab_size, embed_dim)(decoder_inputs)
x = TransformerDecoder(embed_dim, latent_dim, num_heads)(x, encoded_seq_inputs)
x = layers.Dropout(0.5)(x)
decoder_outputs = layers.Dense(vocab_size, activation="softmax")(x)
decoder = keras.Model([decoder_inputs, encoded_seq_inputs], decoder_outputs)

decoder_outputs = decoder([decoder_inputs, encoder_outputs])
transformer = keras.Model(
    [encoder_inputs, decoder_inputs], decoder_outputs, name="transformer"
)

In [40]:
googledrive_path = '/content/drive/MyDrive/Transformers/new_ds_25k/'

In [41]:
from keras import callbacks
early_stopping_cb = callbacks.EarlyStopping(monitor='val_accuracy', patience=4, verbose=True)
checkpoint_cb = callbacks.ModelCheckpoint(googledrive_path+'weights_rmsprop.ckpt', monitor='val_accuracy', save_weights_only=True,verbose=True, save_best_only=True)
tensorboard_callback = callbacks.TensorBoard(log_dir=googledrive_path+"logs")
cbs = [early_stopping_cb, checkpoint_cb, tensorboard_callback]

In [42]:
epochs = 30  # This should be at least 30 for convergence

transformer.summary()
transformer.compile(
    "rmsprop", loss="sparse_categorical_crossentropy", metrics=["accuracy"]
)
transformer.fit(train_ds, epochs=epochs, validation_data=val_ds, callbacks=cbs)

Model: "transformer"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
encoder_inputs (InputLayer)     [(None, None)]       0                                            
__________________________________________________________________________________________________
positional_embedding_2 (Positio (None, None, 256)    6425600     encoder_inputs[0][0]             
__________________________________________________________________________________________________
decoder_inputs (InputLayer)     [(None, None)]       0                                            
__________________________________________________________________________________________________
transformer_encoder_1 (Transfor (None, None, 256)    3155456     positional_embedding_2[0][0]     
________________________________________________________________________________________

In [17]:
latest = tf.train.latest_checkpoint(googledrive_path)
transformer.load_weights(latest)

In [52]:
ar_vocab = ar_vectorization.get_vocabulary()
ar_index_lookup = dict(zip(range(len(ar_vocab)), ar_vocab))
max_decoded_sentence_length = 20


def decode_sequence(input_sentence):
    tokenized_input_sentence = eng_vectorization([input_sentence])
    decoded_sentence = "[start]"
    for i in range(max_decoded_sentence_length):
        tokenized_target_sentence = ar_vectorization([decoded_sentence])[:, :-1]
        predictions = transformer([tokenized_input_sentence, tokenized_target_sentence])

        sampled_token_index = np.argmax(predictions[0, i, :])
        sampled_token = ar_index_lookup[sampled_token_index]
        decoded_sentence += " " + sampled_token

        if sampled_token == "[end]":
            break
    return decoded_sentence


test_eng_texts = [pair[0] for pair in val_pairs]
for _ in range(30):
    input_sentence = random.choice(test_eng_texts[:30])
    translated = decode_sequence(input_sentence)
    print(input_sentence, '\n', translated)
    print('*'*50)

Article 10 
 [start] المادة 10 [end]
**************************************************
19. Articles 247, 248 and 249 shall be replaced by the following: 
 [start] 19 يجب على الأقل لا يمكن أن لا يمكن أن تسمح لصناديق التزامات الاستثمار [end]
**************************************************
amending the Annexes to Council Directives 76/895/EEC, 86/362/EEC, 86/363/EEC and 90/642/EEC as regards the fixing of maximum levels for certain pesticide residues in and on cereals, foodstuffs of animal origin and certain products of plant origin, including fruit and vegetables 
 [start] اللائحة المعدلة رقم 9992001 الصادرة عن المجلس رقم 237790 بتاريخ 14 يونيوحزيران 1990 المتعلق بتعديل النظام في ما يتعلق بالمنتجات
**************************************************
Having regard to Council Regulation (EC) No 2991/94 of 5 December 1994 laying down standards for spreadable fats (1), and in particular Article 8 thereof, 
 [start] مع الأخذ في الاعتبار نظام المجلس المفوضية الأوروبية EEC رقم 5 ديسمبركانون 